#   Антагонистическая игра 5x5

<img width="60%" src="gt13.1.jpg" alt="problem 1 statement" />

`Таблица 13.1` определяет Payoff-матрицу $P$ первого игрока.
Задача состоит в выборе оптимального распределения
$2^{\left\{D_1,\ldots,D_n\right\}}\to [0,1]$ (изоморфно вектору $X\in[0,1]^n$),
$n=5$,
в множестве товаров для первого игрока,
при неизвестном распределении $Y\in[0,1]^5$, выбранном вторым игроком.

Обозначим через $V(X, Y)$
выигрыш первого игрока,
при выборе им распределения $X$
и вторым игроком распределения $Y$.
Другими словами:
$$V(X, Y) = X^\mathrm{T}PY.$$

Тогда критерий оптимальности имеет вид:
$$\min_Y X^\mathrm{T}PY \longrightarrow_X \max.$$

Это выражение можно упростить,
с учётом $y_i\in[0,1]$ и $\sum_i y_k = 1$.

При любом фиксированном $X$ имеют место равенства
$$\min_Y X^\mathrm{T}PY = \min_Y\left[\sum_k y_k X^{\mathrm{T}}P e_k\right] =
\min_i X^{\mathrm{T}}P e_i.$$

Другими словами, если $V_\star$ --- оптимальное решение, то
$$X^{\mathrm{T}}P_{[:,i]} \geq V_\star,\ i=\overline{1,n},$$
где $P_{[:,i]} = P e_i$ --- $i$-й столбец матрицы $P$.

Теперь задачу можно переписать в виде:
$$V\to\max,$$
$$P^{\mathrm{T}} X \geq V,\ \mathbf{1}^{\mathrm{T}}X = 1,\ X\geq 0.$$
Неравенство $P^{\mathrm{T}} X \geq V$ понимается поэлементно,
а $\mathbf{1}$ означает вектор, составленный из единиц.

Переместим $V$ в левую часть уравнения,
введём slack переменные $S = \begin{pmatrix}s_1\\ \vdots \\ s_n\end{pmatrix}\geq 0$
и получим следующую линейную программу:
$$-V\to\inf,$$
$$-V\mathbf{1} - S - P^{\mathrm{T}}X = 0,$$
$$\mathbf{1}^\mathrm{T}X = 1.$$

Эквивалентно:
$$
\left\{
\begin{aligned}
& {-}V\to\inf,\\
&
\left(
\begin{array}{c|c|c}
\begin{matrix}
-1 \\ \vdots \\ -1
\end{matrix}
&
\begin{matrix}
-1 & & 0 \\
   & \ddots & \\
0  & & -1
\end{matrix} &
P^{\mathrm{T}} \\ \hline
\begin{matrix}
0
\end{matrix}
&
\begin{matrix}
0 & \cdots & 0
\end{matrix}
&
\begin{matrix}
1 & \cdots & 1
\end{matrix}
\end{array}
\right)
\begin{pmatrix}
V\\
s_1\\
\vdots\\
s_n\\
x_1\\
\vdots\\
x_n
\end{pmatrix}
=\begin{pmatrix}
0\\
\vdots\\
0\\ \hline
1
\end{pmatrix}
,\\
& V,\ S,\ X \geq 0.
\end{aligned}
\right.$$

Очевидно, второму предприятию соответствует такая же программа
с Payoff-матрицей вида $$1- P^\mathrm{T}$$

In [1]:
P = [
    .5 .5 .4 .5 .21;
    .5 .4 .7 .11 .6;
    .21 .3 .4 .1 .7;
    .3 .6 .7 .3 .2;
    .4 .4 .3 .01 .2
];

In [2]:
function [X, V] = solve_antagonistic (P)
    [m, n] = size(P);
    A = [-ones(n,1), -eye(n), P'; zeros(1,n+1), ones(1,m)];
    b = [zeros(n,1);1];
    c = [1; zeros(size(A,2)-1,1)];
    X = glpk(-c,A,b);
    V = c'*X;
    X = X(n+2:end);
endfunction


[X, V1] = solve_antagonistic(P);
[Y, V_wrt_player_2] = solve_antagonistic(1-P');
X, Y, V1, V_wrt_player_2
TOTAL_VOLUME = V1+V_wrt_player_2

X =

   0.67416
   0.00000
   0.32584
   0.00000
   0.00000

Y =

   0.00000
   0.00000
   0.00000
   0.55056
   0.44944

V1 =  0.36966
V_wrt_player_2 =  0.63034
TOTAL_VOLUME =  1


**Первому предприятию** следует выбрать **товары типов** $1$ и $3$
в соотношении $67.416\%$ к $32.584\%$
или, приблизительно, $2:1$.

**Второму предприятию** следует выбрать **товары типов** $4$ и $5$
в соотношении $55\%$ к $45\%$.

In [3]:
X'*P*Y

ans =  0.36966


In [4]:
Y'*(1-P')*X

ans =  0.63034
